In [1]:
import pygame, os, math, random
import numpy as np

colours = {"cyan": (0, 255, 255, 255),
           "blue": (0, 0, 255, 255),
           "orange": (255, 165, 0, 255),
           "yellow": (255, 255, 0, 255),
           "green": (0, 255, 0, 255),
           "purple": (160, 32, 240, 255),
           "red": (255, 0, 0, 255),
           "white": (250, 250, 250),
           "green2": (165, 255, 81),
            "red2": (255, 76, 76),
            "grey": (140, 140, 140),
            "black": (0, 0, 0)}

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
os.environ['SDL_VIDEO_CENTERED'] = '1'
pygame.init()
width, height = 500, 500
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Orthogonal projection")
clock = pygame.time.Clock()

# plane
theta = 0
phi = 0
psi = 0
a = np.array([0, 0, 0])
w = np.array([1, 0, 0])
u = np.array([0, 1, 0])
v = np.array([0, 0, 1])

def rotate_basis(theta, phi, psi):
    """
    Input: theta, angle of rotation
    Output: basis vectors rotated about y axis by theta, about x axis by phi,
            about z axis by psi
    """
    A = np.array([[np.cos(theta), 0, -np.sin(theta)],
                  [0, 1, 0],
                  [np.sin(theta), 0, np.cos(theta)]])
    B = np.array([[1, 0, 0],
                  [0, np.cos(phi), np.sin(phi)],
                  [0, -np.sin(phi), np.cos(phi)]])
    C = np.array([[np.cos(psi), np.sin(psi), 0],
                  [-np.sin(psi), np.cos(psi), 0],
                  [0, 0, 1]])
    w = C @ B @ A @ np.array([1, 0, 0])
    u = C @ B @ A @ np.array([0, 1, 0])
    v = C @ B @ A @ np.array([0, 0, 1])

    return w, u, v

def project_point(p):
    """
    Input: p = (x, y, z) 3D cartesian coordinate
    Output: (x, y) 2D orthogonal projection onto given plane
    """
    # p = (x, y, z) in cartesian coordinates
    # p = xi + yj + zk
    # u, v, w are the new basis (unit) vectors
    # v is the normal to the view plane
    # translate along v to get p' on the plane (x - a).v = 0
    # p' = p - lambda*v
    # where lambda = (p - a).v
    lam = np.dot(p - a, v) / np.linalg.norm(v)
    if lam > 0:
        visible = True
    else:
        visible = False
    p_prime = p - lam * v
    # p' = (x, y, z) in cartesian coordinates of basis i, j, k
    # change basis into u, v, w
    # p' = x'*w + y'*u + z'*v
    # w and u are vectors in the plane, v normal to the plane
    # planar coordinates are (x', y'), dropping v component to project (z maybe 0 anyway?)
    # x' = p'.w, y' = p'.u
    x = np.dot(p_prime, w)
    y = np.dot(p_prime, u)
    return {'pos': np.array([x, y]), 'visible': visible}

def cart_to_pyg(cart):
    """
    Input: (x, y) coordinates
    Output: (x', y') coordinates to display on screen centered at (0,0)

    Pygame has origin top-left, with x increasing right, y down so transform
    final coordinates to display
    """
    return (width/2 + cart[0], height/2 - cart[1])

def draw_line(p1, p2, colour):
    """
    Input: 2 3D coordinates, colour of line
    Output: draws line between them projected onto surface
    """
    # origin of screen
    origin = project_point(a)['pos']
    # points projected to screen
    s1 = project_point(p1)
    s2 = project_point(p2)
    # only draw if both ends visible
    if s1['visible'] or s2['visible']:
        # relative to origin
        s1['pos'] = s1['pos'] - origin
        s2['pos'] = s2['pos'] - origin
        # change to pygame coordintes to draw to screen
        pygame.draw.line(window, colour, cart_to_pyg(s1['pos']), cart_to_pyg(s2['pos']))

def draw_rectangle(p, width, height, depth, colour):
    """
    Input: 3D point of bottom left, width, height, depth values, colour
    Output: draw wireframe cube onto surface
    """
    x = p[0]
    y = p[1]
    z = p[2]
    lines = [
        # bottom face
        [(x, y, z), (x, y, z + depth)],
        [(x, y, z + depth), (x + width, y, z + depth)],
        [(x + width, y, z + depth), (x + width, y, z)],
        [(x + width, y, z), (x, y, z)],
        # top
        [(x, y + height, z), (x, y + height, z + depth)],
        [(x, y + height, z + depth), (x + width, y + height, z + depth)],
        [(x + width, y + height, z + depth), (x + width, y + height, z)],
        [(x + width, y + height, z), (x, y + height, z)],
        # verticals
        [(x, y, z), (x, y + height, z)],
        [(x, y, z + depth), (x, y + height, z + depth)],
        [(x + width, y, z + depth), (x + width, y + height, z + depth)],
        [(x + width, y, z), (x + width, y + height, z)]
    ]
    for points in lines:
        draw_line(points[0], points[1], colour)


def input(theta, phi, psi):
    """
    Accept user input for movment and rotation of viewing plane and basis
    """
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                pygame.quit()

    keys = pygame.key.get_pressed()
    # rotate about y-axis: left, right
    if keys[pygame.K_LEFT]:
        theta += 0.1
    if keys[pygame.K_RIGHT]:
        theta -= 0.1
    # rotate about x-axis: up, down
    if keys[pygame.K_UP]:
        phi -= 0.1
    if keys[pygame.K_DOWN]:
        phi += 0.1
    # rotate about z-axis: (,),(.)
    if keys[pygame.K_COMMA]:
        psi += 0.1
    if keys[pygame.K_PERIOD]:
        psi -=  0.1
    # move along x axis: a, d
    if keys[pygame.K_a]:
        a[0] -= 5
    if keys[pygame.K_d]:
        a[0] += 5
    # move along y axis: space, shift
    if keys[pygame.K_SPACE]:
        a[1] += 5
    if keys[pygame.K_LSHIFT]:
        a[1] -= 5
    # move along z axis: w, s
    if keys[pygame.K_w]:
        a[2] += 5
    if keys[pygame.K_s]:
        a[2] -= 5
    return theta, phi, psi

# game loop
while True:
    # draw black background
    pygame.draw.rect(window, (0, 0, 0), pygame.Rect(0, 0, width, height))
    # set framerate
    clock.tick(10)
    # accept movement and rotation inputs
    theta, phi, psi = input(theta, phi, psi)
    # rotate basis vectors
    w, u, v = rotate_basis(theta, phi, psi)
    
    # draw rectangles and lines
    draw_rectangle((100, 100, 100), 100, 100, 100, colours["red"])
    draw_rectangle((0, 0, 0), 100, 100, 100, colours["blue"])

    draw_rectangle(a - 5, 10, 10, 10, colours["orange"])

    draw_line((-500,0,0),(500,0,0), colours["red"])
    draw_line((-500,0,50),(500,0,50), colours["orange"])
    draw_line((0,-500,0),(0,500,0), colours["green"])
    draw_line((0,0,-500),(0,0,500), colours["blue"])

    # display
    pygame.event.pump()
    pygame.display.flip()

error: video system not initialized